In [15]:
import pandas as pd
import numpy as np
import time
import copy

ex_pt = pd.read_excel("JSP_dataset.xlsx",sheet_name="Processing Time")
ex_ms  = pd.read_excel("JSP_dataset.xlsx",sheet_name="Machines Sequence")

population_size = int(input("enter population size (default:30) : " ) or 30)
crossorver_rate = float(input("enter crossover rate (default:0.8) : ") or 0.8)
mutation_rate = float(input("enter mutation rate (default:0.2) : ") or 0.2)
mutation_select_rate = float(input("enter muation select rate (default:0.2) : ") or 0.2) #this rate is used to  a kind of mutation,to decide how many genes need to mutation
mutation_gene_num = 
iteration_num = int(input("enter the number of iteration times (default:2000) : ") or 2000)



,order,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
0,J1,1,2,3,4,5,6,7,8,9,10
1,J2,1,3,5,10,4,2,7,6,8,9
2,J3,2,1,4,3,9,6,8,7,10,5
3,J4,2,3,1,5,7,9,8,4,10,6
4,J5,3,1,2,6,4,5,9,8,10,7
5,J6,3,2,6,4,9,10,1,7,5,8
6,J7,2,1,4,3,7,6,10,9,8,5
7,J8,3,1,2,6,5,7,9,10,8,4
8,J9,1,2,4,6,3,10,7,8,5,9
9,J10,2,1,3,7,9,10,6,4,5,8


,order,O1,O2,O3,O4,O5,O6,O7,O8,O9,O10
0,J1,1,2,3,4,5,6,7,8,9,10
1,J2,1,3,5,10,4,2,7,6,8,9
2,J3,2,1,4,3,9,6,8,7,10,5
3,J4,2,3,1,5,7,9,8,4,10,6
4,J5,3,1,2,6,4,5,9,8,10,7
